# Introdução ao Django: Python 

[Aprenda Python com Jupyter](https://github.com/jeanto/python_django_course_notebook) by [Jean Nunes](https://jeanto.github.io/jeannunes)   
Code license: [GNU-GPL v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

---

# Associação entre `classes`

Para implementar a funcionalidade de intenção de doação e seleção de órgãos, vamos precisar criar associação entre classes (Doador e IntencaoDeDoar):

### **1. Atualizar `models.py`** 

Criaremos o modelo `Orgao` para armazenar os nomes dos órgãos e o modelo `IntencaoDeDoar` para incluir um relacionamento _Many-to-Many_ com `Orgao` e um campo booleano para indicar a intenção de doar agora.

```python
# arquivo models.py

from django.db import models
class Orgao(models.Model):
    """
    Modelo para representar os tipos de órgãos que podem ser doados.
    """
    nome = models.CharField(max_length=100, unique=True)

    class Meta:
        verbose_name = "Órgão"
        verbose_name_plural = "Órgãos"

    def __str__(self):
        return self.nome
```

**Model `Orgao`:**

- Um modelo simples com um `CharField` nome para armazenar o nome do órgão (ex: "Coração", "Rins"). `unique=True` garante que não haverá órgãos duplicados.

```python
# arquivo models.py

class IntencaoDeDoar(models.Model):
    doador = models.OneToOneField(
        Doador, 
        on_delete=models.CASCADE, 
        related_name='intencao_doar'
    ) # OneToOneField para garantir 1 intenção por doador

    data_intencao = models.DateField(auto_now_add=True) # Data da criação da intenção
    status = models.CharField(max_length=50, default='ativa') # Ex: 'ativa', 'inativa', 'concluida'
    
    # Novo campo para indicar se o doador tem intenção de doar agora
    doar_agora = models.BooleanField(default=False)
    
    # Novo campo para armazenar os órgãos que o doador deseja doar
    orgaos = models.ManyToManyField(Orgao, blank=True) # blank=True permite que não haja órgãos selecionados

    class Meta:
        verbose_name = "Intenção de Doar"
        verbose_name_plural = "Intenções de Doar"

    def __str__(self):
        return f"Intenção de {self.doador.nome} - Status: {self.status}"
```

**Model `IntencaoDeDoar`**

- `doador = models.OneToOneField(Doador, on_delete=models.CASCADE, related_name='intencao_doar')`: `OneToOneField` garante que cada `Doador` tenha no máximo uma `IntencaoDeDoar` associada.

- `data_intencao = models.DateField(auto_now_add=True)`: A data da intenção será preenchida automaticamente na criação.

- `status = models.CharField(max_length=50, default='ativa')`: Mantido para controle de status.

- `doar_agora = models.BooleanField(default=False)`: Campo para a pergunta "intenção de doar agora".

- `orgaos = models.ManyToManyField(Orgao, blank=True)`: Novo campo para armazenar os órgãos que o doador deseja doar. `blank=True` permite que um doador expresse intenção sem especificar órgãos imediatamente, ou que não queira doar nenhum órgão específico.

Com os modelos definidos, você precisará executar as migrações do Django para criar a nova tabela `Orgao` e atualizar a tabela `IntencaoDeDoar` no seu banco de dados.

```bash
python manage.py makemigrations
python manage.py migrate
```

### **2. Atualizar `forms.py`**

Para popular a tabela `Orgao` com a lista de órgãos, a maneira mais limpa é criar um *"management command"* no Django.

#### 1. Crie o arquivo do comando

- Dentro de `doacoes/management/commands/`, crie um arquivo chamado `populate_orgaos.py`

#### 2. Adicione o código ao `populate_orgaos.py`

In [ ]:
from django.core.management.base import BaseCommand
from sndot.models import Orgao # Importe o seu model Orgao

class Command(BaseCommand):
    help = 'Popula a tabela Orgao com uma lista predefinida de órgãos.'

    def handle(self, *args, **kwargs):
        orgaos_lista = [
            "Coração", "Rins", "Fígado", "Pâncreas", "Pulmões", "Intestino",
            "Córneas", "Pele", "Ossos", "Válvulas cardíacas", "Cartilagem",
            "Medula Óssea", "Tendões", "Vasos Sanguíneos", "Sangue de Cordão Umbilical",
            "Sangue Universal"
        ]

        self.stdout.write(self.style.SUCCESS('Iniciando a população da tabela Orgao...'))

        for nome_orgao in orgaos_lista:
            # get_or_create tenta obter o objeto; se não existe, cria.
            orgao, created = Orgao.objects.get_or_create(nome=nome_orgao)
            if created:
                self.stdout.write(self.style.SUCCESS(f"Órgão '{orgao.nome}' criado com sucesso."))
            else:
                self.stdout.write(self.style.WARNING(f"Órgão '{orgao.nome}' já existe."))

        self.stdout.write(self.style.SUCCESS('População da tabela Orgao concluída.'))


#### 3. Executar o comando

Abra seu terminal na raiz do projeto Django (onde está o `manage.py`) e execute:

```bash
python manage.py populate_orgaos
```

Isso preencherá sua tabela `Orgao` com os dados.

#### 4. Atualizar `forms.py`

Vamos modificar o `CadastrarDoadorForm` no seu `forms.py` para incluir os campos de intenção de doação e seleção de órgãos.


- Importação de Orgao: `from .models import Doador, Orgao`

- Novos Campos no Formulário:

```python
    doar_agora = forms.BooleanField(label="Tenho intenção de doar agora", required=False)
    orgaos_desejados = forms.ModelMultipleChoiceField(
        queryset=Orgao.objects.all(),
        widget=forms.CheckboxSelectMultiple,
        required=False,
        label="Quais órgãos deseja doar?"
    )
```


- `doar_agora = forms.BooleanField(label="Tenho intenção de doar agora", required=False):` Um checkbox simples.
    
- `orgaos_desejados = forms.ModelMultipleChoiceField(...):`

    - `queryset=Orgao.objects.all()`: Ele buscará todas as opções de órgãos do seu modelo `Orgao` (por isso a importância de popular a tabela primeiro).
        
    - `widget=forms.CheckboxSelectMultiple`: Renderiza as opções como uma lista de checkboxes, permitindo múltiplas seleções.

    - `required=False`: Por padrão, não é obrigatório, mas a validação no `clean` o tornará obrigatório se `doar_agora` for `True`.



- Inicialização no `__init__`: A lógica foi adicionada para preencher os valores iniciais de `doar_agora` e `orgaos_desejados` se o formulário estiver sendo usado para editar um doador existente que já possui uma `IntencaoDeDoar` associada.

```python
        # Popula os campos de Intenção de Doar se uma instância de Doador for passada
        # (usado na edição de doador)
        doador_instance = kwargs.get('instance')
        if doador_instance and doador_instance.pk:
            try:
                intencao = doador_instance.intencao_doar
                initial_doar_agora = intencao.doar_agora
                initial_orgaos_desejados = intencao.orgaos.all()
                kwargs['initial'] = kwargs.get('initial', {})
                kwargs['initial']['doar_agora'] = initial_doar_agora
                kwargs['initial']['orgaos_desejados'] = initial_orgaos_desejados
            except doador_instance._meta.model.intencao_doar.RelatedObjectDoesNotExist:
                # Nenhuma intenção de doar existente para este doador
                pass
```

- Validação no `clean()`: Uma nova regra de validação foi adicionada: se `doar_agora` for marcado (`True`), então `orgaos_desejados` se torna obrigatório.

```python
    # Validação condicional para orgaos_desejados
    if doar_agora and not orgaos_desejados:
        self.add_error('orgaos_desejados', 'Por favor, selecione pelo menos um órgão se a intenção de doar agora estiver marcada.')

```

### **3. Atualizar `views.py`**

Ajustaremos a _view_ `cadastrar_doador` para processar esses novos campos e criar/atualizar a instância de `IntencaoDeDoar` após o cadastro do doador.

### **4. Atualizar `cadastrar_doador.html`**

Adicionaremos os novos campos ao formulário HTML e a lógica para exibir/ocultar a lista de órgãos com base na seleção da intenção de doar.